# Output Fixing Parser

- Author: [Jeongeun Lim](https://www.linkedin.com/in/jeongeun-lim-808978188/)
- Peer Review : [Junseong Kim](https://www.linkedin.com/in/%EC%A4%80%EC%84%B1-%EA%B9%80-591b351b2/)
- Proofread : [Two-Jay](https://github.com/Two-Jay)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/08-OutputFixingParser.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/08-OutputFixingParser.ipynb)
## Overview

The ```OutputFixingParser``` in LangChain provides an automated mechanism for correcting errors that may occur during the output parsing process. This parser is designed to wrap around another parser, such as the ```PydanticOutputParser```, and intervenes when the underlying parser encounters outputs that are malformed or do not conform to the expected format. It achieves this by leveraging additional LLM calls to fix the errors and ensure proper formatting.

At its core, the ```OutputFixingParser``` addresses situations where the initial output does not comply with a predefined schema. If such an issue arises, the parser automatically detects the formatting errors and submits a new request to the model, including specific instructions for correcting the issue. These instructions highlight the problem areas and provide clear guidelines for restructuring the data in the correct format.

This functionality is particularly useful in scenarios where strict adherence to a schema is critical. For example, when using the ```PydanticOutputParser``` to generate outputs conforming to a particular data schema, issues such as missing fields or incorrect data types might occur. 

- The ```OutputFixingParser``` steps in as follows:

1. **Error Detection** : It recognizes that the output does not meet the schema requirements.
2. **Error Correction** : It generates a follow-up request to the LLM with explicit instructions to address the issues.
3. **Reformatted Output with Specific Instructions** : The ```OutputFixingParser``` ensures that the correction instructions precisely identify the errors, such as missing fields or incorrect data types. The instructions guide the LLM to reformat the output to meet the schema requirements accurately.


---- 
Practical Example:

Suppose you are using the ```PydanticOutputParser``` to enforce a schema requiring specific fields like ```name``` (string), ```age``` (integer), and ```email``` (string). If the LLM produces an output where the ```age``` field is missing or the ```email``` field is not a valid string, the ```OutputFixingParser``` automatically intervenes. It would issue a new request to the LLM with detailed instructions such as:

- "The output is missing the ```age``` field. Add an appropriate integer value for ```age```."
- "The ```email``` field contains an invalid format. Correct it to match a valid email string."

This iterative process ensures the final output conforms to the specified schema without requiring manual intervention.


---- 
Key Benefits: 

- **Error Recovery**: Automatically handles malformed outputs without requiring user input.
- **Enhanced Accuracy**: Ensures outputs conform to predefined schemas, reducing the risk of inconsistencies.
- **Streamlined Workflow**: Minimizes the need for manual corrections, saving time and improving efficiency.


---- 
Implementation Steps: 

To use the ```OutputFixingParser``` effectively, follow these steps:

1. **Wrap a Parser**: Instantiate the ```OutputFixingParser``` with another parser, such as the ```PydanticOutputParser```, as its base.
2. **Define the Schema**: Specify the schema or format that the output must adhere to.
3. **Enable Error Correction**: Allow the ```OutputFixingParser``` to detect and correct errors automatically through additional LLM calls, ensuring that correction instructions precisely identify and address issues for accurate reformatting.

By integrating the ```OutputFixingParser``` into your workflow, you can ensure robust error handling and maintain consistent output quality in your LangChain applications.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Define Data Model and Set Up PydanticOutputParser](#define-data-model-and-set-up-pydanticoutputparser)
- [Using OutputFixingParser to Correct Incorrect Formatting](#using-outputfixingparser-to-correct-incorrect-formatting)

### References

- [LangChain API Reference](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.fix.OutputFixingParser.html)
- [Pydantic Docs](https://docs.pydantic.dev/latest/api/base_model/)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "08-OutputFixingParser",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it. 

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Define Data Model and Set Up PydanticOutputParser

- The Actor class is defined using the Pydantic model, where name and film_names are fields representing the actor's name and a list of films they starred in.
- The ```PydanticOutputParser``` is used to parse outputs into an Actor object.

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


# Define the Actor class using Pydantic
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


# A query to generate the filmography for a random actor
actor_query = "Generate the filmography for a random actor."

# Use PydanticOutputParser to parse the output into an Actor object
parser = PydanticOutputParser(pydantic_object=Actor)

### Attempt to Parse Misformatted Input Data

- The misformatted variable contains an incorrectly formatted string, which does not match the expected structure (using ' instead of ").
- Calling parser.parse() will result in an error because of the format mismatch.

In [ ]:
# Intentionally input misformatted data
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"
parser.parse(misformatted)

# An error will be printed because the data is incorrectly formatted

## Using OutputFixingParser to Correct Incorrect Formatting
### Set Up OutputFixingParser to Automatically Correct the Error
- ```OutputFixingParser``` wraps around the existing ```PydanticOutputParser``` and automatically fixes errors by making additional calls to the LLM.
- The from_llm() method connects ```OutputFixingParser``` with ```ChatOpenAI``` to correct the formatting issues in the output.

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import OutputFixingParser

# Define a custom prompt to provide the fixing instructions
fixing_prompt = PromptTemplate(
    template=(
        "The following JSON is incorrectly formatted or incomplete: {completion}\n"
    ),
    input_variables=[
        "completion",
    ],
)

# Use OutputFixingParser to automatically fix the error
new_parser = OutputFixingParser.from_llm(
    parser=parser, llm=ChatOpenAI(model="gpt-4o"), prompt=fixing_prompt
)

In [8]:
# Misformatted Output Data
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

### Parse the Misformatted Output Using OutputFixingParser
- The new_parser.parse() method is used to parse the misformatted data. OutputFixingParser will correct the errors in the data and generate a valid Actor object.

In [9]:
actor = new_parser.parse(misformatted)

# Attempt to parse the misformatted JSON with Exception Handling
# try:
#     actor = new_parser.parse(misformatted)
#     print("Parsed actor:", actor)
# except Exception as e:
#     print("Error while parsing:", e)

### Check the Parsed Result
- After parsing, the result is a valid Actor object with the corrected format. The errors in the initial misformatted string have been automatically fixed by OutputFixingParser.

In [10]:
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])